In [3]:
import sys 
sys.path.append('coeus-inference-backend-client-python3')

In [4]:
from coeus_inference_backend_client.client import InferenceBackendClient
from coeus_inference_backend_client.utils import (RunInferenceRequest,
                                                  ModelInput,
                                                  Environment,
                                                  BatchRunInferenceRequest,
                                                  BatchModelInput)

model_name = "query_classify"
namespace = "default-gpu"
env = Environment.DEV
client = InferenceBackendClient(namespace, env)
print(client._host)


coeus-inference-backend-gpu.dev.s.legacy.wish.site:8081


In [5]:
%%timeit
# For now the server side only accepts list with length 1
input_values = ["Generate taxonomy for query: dildo"]

model_input = ModelInput('string_input') # doesn't matter
model_input.extend(input_values)

run_inference_request = RunInferenceRequest(model_name=model_name)
run_inference_request.add_model_input(model_input)

response = client.run_text_generation(run_inference_request)
print(response)

["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
["['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']"]
147 ms ± 8.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
from tqdm import tqdm

In [7]:
import pandas as pd 
df_test = pd.read_json('../datasets/data/query_label/processed/Offshore_Labelled_Query_Classification_Test.json', lines=True)

In [8]:
%%time
res = {}
for i in tqdm(df_test['query'].tolist()):
    input_values = [f"Generate taxonomy for query: {i}"]

    model_input = ModelInput('string_input') # doesn't matter
    model_input.extend(input_values)

    run_inference_request = RunInferenceRequest(model_name=model_name)
    run_inference_request.add_model_input(model_input)

    response = client.run_text_generation(run_inference_request)
    res[i] = eval(response[0])[0].replace('<pad>', '').replace('</s>', '').strip()

100%|██████████| 4300/4300 [10:20<00:00,  6.93it/s]

CPU times: user 7.47 s, sys: 2.06 s, total: 9.53 s
Wall time: 10min 20s


In [21]:
(10 * 60 + 20) / 4300, 4300 / (10 * 60 + 20)

(0.14418604651162792, 6.935483870967742)

In [14]:
df_tax = pd.read_json('../datasets/data/taxonomy/wish_newtax.json', lines=True)
leaf_paths = df_tax[df_tax.is_leaf]['category_path'].tolist()
leaf_paths_lower = [i.lower() for i in leaf_paths]
lower2uppermap = {}
for l, u in zip(leaf_paths_lower, leaf_paths):
    lower2uppermap[l] = u

In [19]:
from thefuzz import process as fuzz_process

In [20]:
%%time
# too slow, would rather do constrained generation than post-hoc fuzzy match
fuzz_process.extractOne(res[i], leaf_paths_lower)

CPU times: user 805 ms, sys: 0 ns, total: 805 ms
Wall time: 803 ms


('watches > watch accessories > watchbands', 100)

In [26]:
import numpy as np

In [27]:
np.mean([v in lower2uppermap for k, v in res.items()])

0.7579069767441861